# Importing the Dependencies

In [51]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# NLTK
- NLTK (Natural Language Toolkit) is a Python library used for working with human language data. 
- A stopword in NLTK refers to commonly used words (like “the”, “is”, “in”) that are often filtered out in text processing to focus on more meaningful words.

In [91]:
# printing the stopwords in English
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\RASHIF02\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Data Pre-processing
- training dataset with the following attributes:

    - id: unique id for a news article
    - title: the title of a news article
    - author: author of the news article
    - text: the text of the article; could be incomplete
    - label: a label that marks the article as potentially unreliable
        - 1: unreliable
        - 0: reliable

- https://www.kaggle.com/competitions/fake-news/data?select=train.csv


In [92]:
# loading the dataset to a pandas DataFrame
fakeNews_data = pd.read_csv('Dataset/fakeNews.csv')
fakeNews_data.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [54]:
fakeNews_data.shape

(20800, 5)

In [55]:
# counting the number of missing values in the dataset
fakeNews_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [56]:
# replacing the null values with empty string
fakeNews_data = fakeNews_data.fillna('')
fakeNews_data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [70]:
# merging the author name and news title
fakeNews_data['content'] = fakeNews_data['author'] + ' ' + fakeNews_data['title']
fakeNews_data['content']

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

In [71]:
# separating the data & label
X = fakeNews_data.drop(columns='label', axis=1)
Y = fakeNews_data['label']

In [72]:
print(f'Training Set:\n {X}')

Training Set:
           id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                       

In [73]:
print(f'Label:\n {Y}')

Label:
 0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


# Stemming
- Stemming in NLP is the process of reducing words to their base or root form. For example, “running” becomes “run”. This helps in normalizing text for analysis.

- The PorterStemmer is a popular stemming algorithm in NLTK that applies a series of rules to transform words into their stems.

In [74]:
port_stem = PorterStemmer()

In [93]:
# This function, stemming, processes a given text (content) by performing the following steps:
# Remove non-alphabetic characters: It replaces any character that is not a letter with a space.
# Convert to lowercase: It changes all characters to lowercase.
# Split into words: It splits the text into individual words.
# Stem words and remove stopwords: It stems each word using port_stem (a PorterStemmer instance) and removes common English stopwords.
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [76]:
# apply the stemming function to each element in the content column of the fakeNews_data DataFrame 
fakeNews_data['content'] = fakeNews_data['content'].apply(stemming)

In [80]:
print(fakeNews_data['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [85]:
# separating the data and label
X = fakeNews_data['content']
Y = fakeNews_data['label']
print(X)
print(Y)

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object
0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


- The .values attribute in a pandas DataFrame or Series returns the underlying data as a NumPy array. 
- This can be useful for various operations, especially when you need to interface with libraries that require NumPy arrays.

In [84]:
X = fakeNews_data['content'].values
Y = fakeNews_data['label'].values
print(X)
print(Y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


In [86]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [87]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [88]:
model = LogisticRegression()

In [89]:
model.fit(x_train, y_train)
y_pred_train = model.predict(x_train)
training_data_accuracy = accuracy_score(y_train, y_pred_train)
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9871995192307692


In [90]:
model.fit(x_test, y_test)
y_pred_test = model.predict(x_test)
testing_data_accuracy = accuracy_score(y_test, y_pred_test)
print('Accuracy score of the testing data : ', testing_data_accuracy)

Accuracy score of the testing data :  0.9757211538461539
